# Fine-tune a pre-trained model using Afrikaans/isiXhosa data
### Author: Lucas Meyer

Python notebook used to load a pre-trained model from HuggingFace and fine-tune the model using a HuggingFace dataset (or a custom dataset).

**Important Note**: that we recommend that you should upload this notebook (along with the three ``load_<>.py`` scripts, ``requirements.txt``, and ``utils.py``) to a Google Colab session. Make sure to use a T4 GPU at least, and if you do not have access to one then create a Google burner account so that you have immediate access. :)

**Acknowledgements**: I would like to thank Patrick von Platen, for his super useful [blog-post](https://huggingface.co/blog/fine-tune-xlsr-wav2vec2) on how to fine-tune XLS-R models.

### 1. Install dependencies

Replace "awe" with your sudo password if necessary.

In [1]:
# Run with sudo or not
sudo_password = False
if sudo_password:
    !echo "awe" > password.txt

In [2]:
# Install Python dependencies
!pip3 install -r requirements.txt

# Install GitLFS
if sudo_password:
    !sudo -S apt-get update < password.txt
    !sudo apt-get install git-lfs tree
else:
    !apt-get update
    !apt-get install git-lfs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 105.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.4/427.4 kB 46.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing m

### 2. Setup experiment

**Important Note**: Change ``WRITE_ACCESS_TOKEN`` to your HF token in ``utils.py``

### 2.1 Choose dataset

In [3]:
dataset_name = str(input("Enter name of dataset (asr_af OR asr_xh OR fleurs_zu): "))

Enter name of dataset (asr_af OR asr_xh OR fleurs_zu): asr_af


### 2.2 Choose pre-trained model and repo names

In [10]:
repo_exists = True if (input("Repo exists: ") == "y") else False
sequential = True if (input("Sequential fine-tune: ") == "y") else False

if not repo_exists:
    if not sequential:
        run_number = int(input("Provide the run number please: "))
        PUSH_REPO = f"xls-r-{dataset_name}-run{run_number}"
        PULL_REPO = "facebook/wav2vec2-xls-r-300m"
    else:
        PULL_REPO = input("Provide the pull repo please: ")
        run_number = int(input("Provide the run number please: "))
        first_part = PULL_REPO.split("/")[1]
        PUSH_REPO = f"seq-{first_part}-{dataset_name}-run{run_number}"
else:
    PULL_REPO = input("Provide the pull repo please: ")
    PUSH_REPO = PULL_REPO.split("/")[1]

Repo exists: y
Sequential fine-tune: y
Provide the pull repo please: lucas-meyer/wav2vec2-xls-r-300m-fleurs_zu-run1-seq-asr_af


### 3. Import libraries

In [11]:
# Standard imports
import os
import json
import torch
import evaluate
import numpy as np
import pandas as pd
from dataclasses import dataclass
from typing import Dict, List, Union

# My own imports
from load_nchlt import load_nchlt
from load_fleurs import load_fleurs
from load_fleurs_nl import load_fleurs_nl
from load_fleurs_zu import load_fleurs_zu
from load_high_quality_tts import load_high_quality_tts
from utils import (
    SR, WRITE_ACCESS_TOKEN,
    clear_cache,
    remove_special_characters_batch
)

# HuggingFace imports
from datasets import Audio
from datasets import load_dataset
from huggingface_hub import login
from huggingface_hub import Repository
from transformers import Wav2Vec2ForCTC
from transformers import Wav2Vec2Processor
from transformers import Wav2Vec2CTCTokenizer
from transformers import Wav2Vec2FeatureExtractor
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback

### 4. ASR model class

I made this for my own convenience.

In [12]:
class ASR_MODEL:
    def __init__(self, repo_name, dataset_name, load_from_hf, push_dataset, push_repo, write_audio):
        self.repo_name = repo_name
        self.dataset_name = dataset_name
        self.load_from_hf = load_from_hf
        self.push_dataset = push_dataset
        self.push_repo = push_repo
        self.write_audio = write_audio
        os.makedirs(repo_name, exist_ok=True)

        self.train_set = None
        self.val_set = None
        self.test_set = None
        self.tokenizer = None
        self.feature_extractor = None
        self.processor = None
        self.data_collator = None
        self.xlsr_model = None
        self.trainer = None

        if sequential:
            self.xlsr_model = Wav2Vec2ForCTC.from_pretrained(PULL_REPO)
            self.processor = Wav2Vec2Processor.from_pretrained(PULL_REPO)
            self.tokenizer = Wav2Vec2CTCTokenizer.from_pretrained(PULL_REPO)

            self.xlsr_model.freeze_feature_encoder()          # Freeze feature exctraction weights
            self.xlsr_model.gradient_checkpointing_enable()   # Enable gradient checkpointing

    def load_datasets(self):
        self.dataset_dir = os.path.join("data", "speech_data", self.dataset_name)
        if not self.load_from_hf:
            if not os.path.exists(self.dataset_dir):
                os.makedirs(self.dataset_dir, exist_ok=True)
                # Create dataset by combining 3 datasets into an audiofolder
                csv_entries = []
                if (self.dataset_name == "asr_af"):
                    csv_entries += load_fleurs(language="af", write_audio=self.write_audio)
                    csv_entries += load_high_quality_tts(language="af", write_audio=self.write_audio)
                    # csv_entries += load_nchlt(language="af", write_audio=self.write_audio)
                elif (self.dataset_name == "asr_xh"):
                    csv_entries += load_fleurs(language="xh", write_audio=self.write_audio)
                    csv_entries += load_high_quality_tts(language="xh", write_audio=self.write_audio)
                    csv_entries += load_nchlt(language="xh", write_audio=self.write_audio)
                elif (self.dataset_name == "asr_af_xh"):
                    csv_entries += load_fleurs(language="both", write_audio=self.write_audio)
                    csv_entries += load_high_quality_tts(language="both", write_audio=self.write_audio)
                    csv_entries += load_nchlt(language="both", write_audio=self.write_audio)
                elif (self.dataset_name == "fleurs_nl"):
                    csv_entries += load_fleurs_nl(write_audio=self.write_audio, plot_durations=True)
                elif (self.dataset_name == "fleurs_zu"):
                    csv_entries += load_fleurs_zu(write_audio=self.write_audio, plot_durations=True)
                metadata = pd.DataFrame(csv_entries, columns=['file_name', 'transcription'])
                metadata.to_csv(path_or_buf=os.path.join(self.dataset_dir, "metadata.csv"), sep=",", index=False)

                # Load dataset from audiofolder that you created
                dataset = load_dataset("audiofolder", data_dir=self.dataset_dir)

                # Push dataset to huggingface hub
                if self.push_dataset:
                    dataset.push_to_hub(f"lucas-meyer/{self.dataset_name}")
                    # done = False
                    # num_restarts = 0
                    # while not done:
                    #     try:
                    #         dataset.push_to_hub(f"lucas-meyer/{self.dataset_name}")
                    #         done = True
                    #     except Exception as e:
                    #         num_restarts += 1
                    #         print(f"{str(e)}")
                    #         print(f"Restarting (num restarts: {num_restarts})")
            else:
                # Load dataset from audiofolder that you created
                dataset = load_dataset("audiofolder", data_dir=self.dataset_dir)
        else:
            # Load dataset from huggingface hub
            dataset = load_dataset(f"lucas-meyer/{self.dataset_name}")

        # Downsample audio to SR = 16000 and init train/val/test sets
        self.train_set = dataset["train"].cast_column("audio", Audio(sampling_rate=SR)).rename_column("transcription", "sentence")
        self.val_set = dataset["validation"].cast_column("audio", Audio(sampling_rate=SR)).rename_column("transcription", "sentence")
        self.test_set = dataset["test"].cast_column("audio", Audio(sampling_rate=SR)).rename_column("transcription", "sentence")
        self.train_set = self.train_set.map(remove_special_characters_batch)
        self.val_set = self.val_set.map(remove_special_characters_batch)
        self.test_set = self.test_set.map(remove_special_characters_batch)
        torch.cuda.empty_cache()

    def extract_all_chars(self, batch):
        all_text = " ".join(batch["sentence"])
        vocab = list(set(all_text))
        return {"vocab": [vocab], "all_text": [all_text]}

    def create_tokenizer(self):
        vocab_train = self.train_set.map(self.extract_all_chars,
                                         batched=True, batch_size=-1,
                                         keep_in_memory=True,
                                         remove_columns=self.train_set.column_names)

        vocab_val = self.val_set.map(self.extract_all_chars,
                                     batched=True, batch_size=-1,
                                     keep_in_memory=True,
                                     remove_columns=self.val_set.column_names)

        vocab_test = self.test_set.map(self.extract_all_chars,
                                       batched=True, batch_size=-1,
                                       keep_in_memory=True,
                                       remove_columns=self.test_set.column_names)

        # Get list for vocab of train/val/test
        vocab_list = list(set(vocab_train["vocab"][0]) |
                        set(vocab_test["vocab"][0]) |
                        set(vocab_val["vocab"][0]))

        # Get dict for vocab of train/val/test
        vocab_dict = {v: k for k, v in enumerate(sorted(vocab_list))}
        vocab_dict["|"] = vocab_dict[" "]
        del vocab_dict[" "]
        vocab_dict["[UNK]"] = len(vocab_dict)
        vocab_dict["[PAD]"] = len(vocab_dict)

        # Save vocabulary file
        with open(os.path.join(self.repo_name, 'vocab.json'), 'w') as vocab_file:
            json.dump(vocab_dict, vocab_file)

        self.tokenizer = Wav2Vec2CTCTokenizer(os.path.join(self.repo_name, 'vocab.json'),
                                              unk_token="[UNK]",
                                              pad_token="[PAD]",
                                              bos_token=None,
                                              eos_token=None,
                                              word_delimiter_token="|")
        if self.push_repo:
            self.tokenizer.push_to_hub(f"lucas-meyer/{self.repo_name}")

    def prepare_dataset(self, batch):
        audio = batch["audio"]
        batch["input_values"] = self.processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
        batch["input_length"] = len(batch["input_values"])
        batch["labels"] = self.processor(text=batch["sentence"]).input_ids
        return batch

    def create_processor(self):
        self.feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1,
                                                     sampling_rate=16000,
                                                     padding_value=0.0,
                                                     do_normalize=True,
                                                     return_attention_mask=True)

        self.processor = Wav2Vec2Processor(feature_extractor=self.feature_extractor,
                                      tokenizer=self.tokenizer)

    def extract_features(self):
        self.train_set = self.train_set.map(self.prepare_dataset, remove_columns=self.train_set.column_names)
        self.val_set = self.val_set.map(self.prepare_dataset, remove_columns=self.val_set.column_names)
        self.test_set = self.test_set.map(self.prepare_dataset, remove_columns=self.test_set.column_names)

    def create_data_collator(self):
        self.data_collator = DataCollatorCTCWithPadding(processor=self.processor, padding=True)

    def download_xlsr(self):
        self.xlsr_model = Wav2Vec2ForCTC.from_pretrained(
            PULL_REPO,
            attention_dropout=0.0,
            hidden_dropout=0.0,
            feat_proj_dropout=0.0,
            mask_time_prob=0.05,
            layerdrop=0.0,
            ctc_loss_reduction="mean",
            pad_token_id=self.processor.tokenizer.pad_token_id,
            vocab_size=len(self.processor.tokenizer),
        )
        self.xlsr_model.freeze_feature_encoder()          # Freeze feature exctraction weights
        self.xlsr_model.gradient_checkpointing_enable()   # Enable gradient checkpointing

    def compute_metrics(self, pred):
        pred_logits = pred.predictions
        pred_ids = np.argmax(pred_logits, axis=-1)
        pred.label_ids[pred.label_ids == -100] = self.processor.tokenizer.pad_token_id
        pred_str = self.processor.batch_decode(pred_ids)
        # we do not want to group tokens when computing the metrics
        label_str = self.processor.batch_decode(pred.label_ids, group_tokens=False)

        wer_metric = evaluate.load("wer")
        wer = wer_metric.compute(predictions=pred_str, references=label_str)
        return {"wer": wer}

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
            sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
            maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
            different lengths).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )

        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)
        batch["labels"] = labels
        return batch

### 5. Log in to Hugging Face

In [13]:
login(token=WRITE_ACCESS_TOKEN)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


### 6. Instantiate ASR model object

In [ ]:
if dataset_name == "fleurs_zu":
    model = ASR_MODEL(repo_name=PUSH_REPO,
                      dataset_name=dataset_name,
                      load_from_hf=False,
                      push_dataset=False,
                      push_repo=True,
                      write_audio=True)
else:
    model = ASR_MODEL(repo_name=PUSH_REPO,
                      dataset_name=dataset_name,
                      load_from_hf=True,
                      push_dataset=False,
                      push_repo=True,
                      write_audio=False)

### 7. Prepare ASR model for training

In [ ]:
if repo_exists:
    !rm -rf {PUSH_REPO}
    repo = Repository(local_dir=PUSH_REPO, clone_from=PULL_REPO)
    model.load_datasets()
    model.xlsr_model = Wav2Vec2ForCTC.from_pretrained(PUSH_REPO)
    model.processor = Wav2Vec2Processor.from_pretrained(PUSH_REPO)
    model.tokenizer = Wav2Vec2CTCTokenizer.from_pretrained(PUSH_REPO)
    model.xlsr_model.freeze_feature_encoder()          # Freeze feature exctraction weights
    model.xlsr_model.gradient_checkpointing_enable()   # Enable gradient checkpointing
    model.extract_features()
    model.create_data_collator()
elif sequential:
    model.load_datasets()
    model.create_tokenizer()
    model.create_processor()
    model.extract_features()
    model.create_data_collator()
else:
    model.load_datasets()
    model.create_tokenizer()
    model.create_processor()
    model.extract_features()
    model.create_data_collator()
    model.download_xlsr()

### 8. Train ASR model

Useful links:
 - [Training arguments](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.TrainingArguments)

In [ ]:
clear_cache()

training_args = TrainingArguments(
    # Where to save the model
    push_to_hub=True,
    overwrite_output_dir=True,
    output_dir=model.repo_name,

    # Batch sizes and num accumulation steps
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=3,

    # Enable gradient checkpointing and half-point precision
    gradient_checkpointing=True,
    fp16=True,

    # Evaluation stuff
    eval_steps=250,
    evaluation_strategy="steps",

    # Saving stuff
    save_steps=250,
    save_total_limit=2,
    save_strategy="steps",
    load_best_model_at_end=True,

    # Other hyperparameters
    warmup_steps=500,
    learning_rate=3e-4,
    num_train_epochs=30,
    group_by_length=True,
)

model.trainer = Trainer(
    model=model.xlsr_model,
    data_collator=model.data_collator,
    args=training_args,
    compute_metrics=model.compute_metrics,
    train_dataset=model.train_set,
    eval_dataset=model.val_set,
    tokenizer=model.processor.feature_extractor,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

model.trainer.train()
model.trainer.push_to_hub()